In [2]:
import numpy as np
from scipy import linalg

In [3]:
def lu(A):
    
    #Get the number of rows
    n = A.shape[0]
    
    U = A.copy()
    L = np.eye(n, dtype=np.double)
    
    #Loop over rows
    for i in range(n):
            
        #Eliminate entries below i with row operations 
        #on U and reverse the row operations to 
        #manipulate L
        factor = U[i+1:, i] / U[i, i]
        L[i+1:, i] = factor
        U[i+1:] -= factor[:, np.newaxis] * U[i]
        
    return L, U


In [4]:
A = np.arange(1,10,dtype=np.double).reshape(3,3)

In [5]:
lu(A)[0]

array([[1., 0., 0.],
       [4., 1., 0.],
       [7., 2., 1.]])

In [6]:
lu(A)[1]

array([[ 1.,  2.,  3.],
       [ 0., -3., -6.],
       [ 0.,  0.,  0.]])

In [7]:
def plu(A):
    
    #Get the number of rows
    n = A.shape[0]
    
    #Allocate space for P, L, and U
    U = A.copy()
    L = np.eye(n, dtype=np.double)
    P = np.eye(n, dtype=np.double)
    
    #Loop over rows
    for i in range(n):
        
        #Permute rows if needed
        for k in range(i, n-1): 
            if ~np.isclose(U[i, i], 0.0):
                break
            U[[k, k+1]] = U[[k+1, k]]
            P[[k, k+1]] = P[[k+1, k]]
            
        #Eliminate entries below i with row 
        #operations on U and #reverse the row 
        #operations to manipulate L
        factor = U[i+1:, i] / U[i, i]
        L[i+1:, i] = factor
        U[i+1:] -= factor[:, np.newaxis] * U[i]
        
    return P, L, U


In [8]:
plu(A)[0]

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

In [9]:
plu(A)[1]

array([[1., 0., 0.],
       [4., 1., 0.],
       [7., 2., 1.]])

In [10]:
plu(A)[2]

array([[ 1.,  2.,  3.],
       [ 0., -3., -6.],
       [ 0.,  0.,  0.]])

In [11]:
def forward_substitution(L, b):
    
    #Get number of rows
    n = L.shape[0]
    
    #Allocating space for the solution vector
    y = np.zeros_like(b, dtype=np.double);
    
    #Here we perform the forward-substitution.  
    #Initializing  with the first row.
    y[0] = b[0] / L[0, 0]
    
    #Looping over rows in reverse (from the bottom  up),
    #starting with the second to last row, because  the 
    #last row solve was completed in the last step.
    for i in range(1, n):
        y[i] = (b[i] - np.dot(L[i,:i], y[:i])) / L[i,i]
        
    return y


In [12]:
def back_substitution(U, y):
    
    #Number of rows
    n = U.shape[0]
    
    #Allocating space for the solution vector
    x = np.zeros_like(y, dtype=np.double);

    #Here we perform the back-substitution.  
    #Initializing with the last row.
    x[-1] = y[-1] / U[-1, -1]
    
    #Looping over rows in reverse (from the bottom up), 
    #starting with the second to last row, because the 
    #last row solve was completed in the last step.
    for i in range(n-2, -1, -1):
        x[i] = (y[i] - np.dot(U[i,i:], x[i:])) / U[i,i]
        
    return x


In [13]:
def lu_solve(A, b):
    
    L, U = lu(A)
    
    y = forward_substitution(L, b)
    
    return back_substitution(U, y)

In [14]:
b = np.array([1,3,1],dtype=np.double)
A_1 = np.array([[1,4,1],[2,1,3],[2,4,1]],dtype=np.double)

In [15]:
lu_solve(A_1, b)

array([ 2.22044605e-16, -1.58603289e-17,  1.00000000e+00])

In [16]:
def plu_solve(A, b):
    
    P, L, U = plu(A)
    
    y = forward_substitution(L, np.dot(P, b))
    
    return back_substitution(U, y)


In [17]:
plu_solve(A_1, b)

array([ 2.22044605e-16, -1.58603289e-17,  1.00000000e+00])

In [18]:
linalg.solve(A_1,b)

array([0., 0., 1.])

In [22]:
A_2 = np.array([[-3,2,-4],[0,1,2],[2,4,5]],dtype=np.double)
b_2 = np.array([12,5,2],dtype=np.double)

In [23]:
lu_solve(A_2, b_2)

array([-6.,  1.,  2.])

In [24]:
linalg.solve(A_2,b_2)

array([-6.,  1.,  2.])

In [25]:
plu_solve(A_2, b_2)

array([-6.,  1.,  2.])

In [26]:
# PLU лучше чем LU когда на главной диагональ стоит 0 для некоторые итдекси матрица  

In [29]:
A_3 = np.array([[-3,2,-4],[0,0,2],[2,4,5]],dtype=np.double)

In [34]:
lu(A_3)[0] #L

C:\Users\lenovo\python3\lib\site-packages\ipykernel_launcher.py:15: RuntimeWarning: divide by zero encountered in true_divide
  from ipykernel import kernelapp as app
C:\Users\lenovo\python3\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in multiply


array([[ 1.        ,  0.        ,  0.        ],
       [-0.        ,  1.        ,  0.        ],
       [-0.66666667,         inf,  1.        ]])

In [35]:
lu(A_3)[1]#U inf, pazdelenie na nuli

C:\Users\lenovo\python3\lib\site-packages\ipykernel_launcher.py:15: RuntimeWarning: divide by zero encountered in true_divide
  from ipykernel import kernelapp as app
C:\Users\lenovo\python3\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in multiply


array([[ -3.,   2.,  -4.],
       [  0.,   0.,   2.],
       [ nan,  nan, -inf]])

In [36]:
plu(A_3)[0] #перестановка матрица , показывает что мы помняем строк

array([[1., 0., 0.],
       [0., 0., 1.],
       [0., 1., 0.]])

In [37]:
plu(A_3)[1] # L

array([[ 1.        ,  0.        ,  0.        ],
       [-0.        ,  1.        ,  0.        ],
       [-0.66666667,  0.        ,  1.        ]])

In [39]:
plu(A_3)[2] # U

array([[-3.        ,  2.        , -4.        ],
       [ 0.        ,  5.33333333,  2.33333333],
       [ 0.        ,  0.        ,  2.        ]])